# WaveNet

1. Architecture with stacked 1D convolutions
2. Those convolutional layers have a delation rate - that describes how far the neurons are apart from each other. 
3. Enables us to process large sequences of data

Important: 
1. The low level layers learn low level features 
2. The high level layers learn long-term patterns



<img src = "../../img/1511.png" />

### Dialeted Convolutions

<img src = "../../img/9999.gif" />

1. The _dialation rate_ defines the values in the kernel/filter. 
2. The _dialation rate_ is more broader field of reception
3. A 3x3 kernel with a dialation rate of 2 will have a field of view as 5x5

__Benefit__: 
- Wider field of reception with the same computational costs

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Training Data

In [3]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1) # Creates an array of given shape (4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  #   wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)   # + noise
    return series[..., np.newaxis].astype(np.float32)

In [4]:
n_steps = 50
series = generate_time_series(10000, n_steps+1) # n_steps +1 generate us a label

In [5]:
series.shape # (batch_size, time steps, dimensionality)

(10000, 51, 1)

In [6]:
x_train, y_train = series[:7000, :n_steps], series[:7000, -1] # -1 takes the last time series step

In [7]:
y_train

array([[ 0.52530134],
       [-0.30187267],
       [-0.1694343 ],
       ...,
       [-0.3248322 ],
       [ 0.45576847],
       [-0.08068113]], dtype=float32)

In [8]:
x_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]

In [9]:
x_test, y_test = series[9000:, :n_steps], series[:9000, -1]

# WaveNet

In [12]:
def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])


In [13]:
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape = [None, 1]))
for rate in (1,2,4,8)*2:
    model.add(keras.layers.Conv1D(filters=20, kernel_size=2, padding="causal", activation="relu", dilation_rate=rate))
model.add(keras.layers.Conv1D(filters=1, kernel_size=1))

In [14]:
model.compile(loss="mse", optimizer="adam", metrics = [last_time_step_mse])

In [16]:
model.fit(x_train, y_train, epochs=20, validation_data=(x_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 1s 7ms/step - loss: 0.1127 - last_time_step_mse: 0.1473 - val_loss: 0.1178 - val_last_time_step_mse: 0.1540
Epoch 2/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1118 - last_time_step_mse: 0.1481 - val_loss: 0.1157 - val_last_time_step_mse: 0.1522
Epoch 3/20
219/219 [==============================] - 1s 7ms/step - loss: 0.1111 - last_time_step_mse: 0.1489 - val_loss: 0.1163 - val_last_time_step_mse: 0.1535
Epoch 4/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1100 - last_time_step_mse: 0.1483 - val_loss: 0.1166 - val_last_time_step_mse: 0.1525
Epoch 5/20
219/219 [==============================] - 1s 7ms/step - loss: 0.1096 - last_time_step_mse: 0.1491 - val_loss: 0.1163 - val_last_time_step_mse: 0.1585
Epoch 6/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1093 - last_time_step_mse: 0.1491 - val_loss: 0.1139 - val_last_time_step_mse: 0.1596
Epoch 7/20
219/219 [========